# Description 

Integrates a single trajectory with $n=20$, $k=1$ (nearest neighbors), starting with a $q=3$ twisted state.

In [ ]:
using Plots
using Printf
using Random
using LaTeXStrings
using DSP
using LinearAlgebra
using ForwardDiff
using NLsolve
using Plots.Measures

In [ ]:
default(xtickfontsize=14,  ytickfontsize=14, 
    guidefontsize=14, 
    legendfontsize=12, lw=2,ms=4)

In [ ]:
push!(LOAD_PATH, "../GeneralizedKuramoto");
push!(LOAD_PATH, "../QProcesses");
push!(LOAD_PATH, "../GraphMatrices");
push!(LOAD_PATH, "../TwistedStates");
push!(LOAD_PATH, "../ClassicalKuramoto");

In [ ]:
using GeneralizedKuramoto
using QProcesses
using GraphMatrices
using TwistedStates
using ClassicalKuramoto

# Parameter choices

In [ ]:
n = 20;     # number of sites
k = 1;      # interaction range, nearest neighbors
q = 3;      # initial twisted state
Δt = 1e-2;  # tiem step
α = 0;      # attractive coupling
σ = 0.2;    # noise intensity


# Prepare data

In [ ]:

u₀ = TwistedStates.construct_q_twisted(n, q);
u = mod.(copy(u₀), 1);

# Set solver for integration

In [ ]:
# prepare interaction matrix
K = 1/k * GraphMatrices.discrete_knn_spmatrix(n,k);

# prepare sampler
Q_sampler = let σ = σ, n = n
    ()-> QProcesses.IsoGaussian(σ,n)
end

# prepare nonlinearity
f = (t,u)-> 0.0;
# this si the repulsive coupling
S = let α = α
    (u,v)-> (-1)^(α) * sinpi(2 *(v-u));
end

E = u-> ClassicalKuramoto.energy(u,K,α);
∇E = u->ForwardDiff.gradient(E,u);
HessE = u->ForwardDiff.hessian(E,u);

# set time stepping parameters
tmax = 1e3;
Δt_save = 1e-2;
@show nΔt = Int(tmax/Δt);
@show nsave = Int(Δt_save/Δt);
# nsave = 1; # save every nsave frames


In [ ]:
@show E(u₀);
@show eigmin(HessE(u₀));

In [ ]:
scatter(1:n, u₀,label="IC")

# Integrate the trajectory

In [ ]:
Random.seed!(100);
t_trajectory, u_trajectory = GeneralizedKuramoto.EulerMaruyama(u₀, f, K, 
    S, Q_sampler, 
    Δt, nΔt, nsave = nsave, 
    verbose=false);

# Visualize transitions

In [ ]:
#
idx_vals = [1, 101, 175, 236, 271, 501];

plot(t_trajectory[1:1:end],E.(u_trajectory[1:1:end]),label="")
scatter!(t_trajectory[idx_vals], [E(u_trajectory[i]) for i in idx_vals],
    label="", ms = 6)

pt_labels = ["(i)", "(ii)", "(iii)", "(iv)", "(v)", "(vi)"]
pt_xoffsets = [0.3, 0.2,0.3, -0.1, 0.3, 0.2, 0.2];
pt_yoffsets = [-0.1, -0.1, -0.1, -0.1, 0.1, +0.2, -0.2];
for (i,idx) in enumerate(idx_vals)
    annotate!(t_trajectory[idx]+pt_xoffsets[i], E(u_trajectory[idx])+pt_yoffsets[i], (pt_labels[i],:red))
end
xlabel!(L"$t$")
ylabel!("Energy")
xlims!(0,6)


In [ ]:
#
idx_vals = [1, 101, 175, 236, 271, 501,50001+1065,90001];

plot(t_trajectory[1:1:end],E.(u_trajectory[1:1:end]),label="", 
    xaxis=(:log10, [1e-2, :auto]), right_margin = 2 *Plots.mm)
scatter!(t_trajectory[idx_vals], [E(u_trajectory[i]) for i in idx_vals],
    label="", ms = 6)
pt_labels = ["(ii)", "(iii)", "(iv)", "(v)", "(vi)", "(vii)", "(viii)"]
pt_xoffsets = [0.2, 1, -0.4, 1.8, 0.2, 0.2, 0.2];
pt_yoffsets = [-0.1, -0.1, -0.1, -0.1, -0.2, +0.1, -0.2];
for (i,idx) in enumerate(idx_vals[2:end])
    annotate!(t_trajectory[idx]+pt_xoffsets[i], E(u_trajectory[idx])+pt_yoffsets[i], (pt_labels[i],:red))
end
xlabel!(L"$t$")
ylabel!("Energy")

Visualize the particular farmes associated with the points in the energy time series.  Change the value of `idx` to see different points in the trajectory.

In [ ]:
idx_vals = [1, 101, 175, 236, 271, 501,50001+1065,90001];
idx = idx_vals[3]
scatter((0:n-1), mod.(u_trajectory[idx].-u_trajectory[idx][1],1),label="")
title!(latexstring(@sprintf("t = %.2f", t_trajectory[idx])));
ylims!(-0.1, 1.1)
